In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import altair as alt # graphical representation tool
import datetime as dt
alt.renderers.enable('kaggle')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
netflix_movies = pd.read_csv('../input/netflix-shows/netflix_titles.csv')
netflix_stocks = pd.read_csv('../input/netflix-stock-data/NFLX.csv')

In [ ]:
netflix_movies = netflix_movies[netflix_movies['type']=='Movie'].dropna()
netflix_movies.head()

In [ ]:
netflix_stocks.head()

Now, that we have our DataFrames ready; we can continue to doing some investigative work. Let's check the descriptive statistics for both stocks and a couple different segmentations for movies with pd.DataFrame.describe().

In [ ]:
netflix_movies.date_added = pd.to_datetime(netflix_movies.date_added)
movies_over_time = netflix_movies.copy().groupby('release_year')['title'].count().reset_index()
movies_over_time = movies_over_time.rename(columns={0:'count'})

median_year = int(netflix_movies ['release_year'].median()) #2016
mode_year = int(netflix_movies ['release_year'].mode())
print("this is the median:", median_year, ", this is the mode:", mode_year)
movies_over_time.describe()

In [ ]:
netflix_stocks['Date'] = pd.to_datetime(netflix_stocks['Date'])

netflix_stocks.describe()

We get to see our maxes and mins & highs and lows. We have a pretty good idea of the distribution for the movies and the average closing dollar amount. Could Netflix releases over the years correlated with the stock price? Can we tell if our answer is statistically significant?

If you think about releases, there are four elements of releases that might catch a movie lover's attention. You have:
1. Main film location:  language matters but relativity to the elements in the movie and being familiar with the references is quentissential in most instances. 
2. Director: we all have our favorite director. It's the movie that was most memorable or the one that made you laugh the most. You loved the cinamatic touches they added. You strap in for a good movie when you hear what's on for movie night.
3. Actor: Leonardo DiCaprio gets my buy in on any movie. We all have that person or a few! Is their a primary actor driving the market?
4. Genre: I love a good horror movie, don't you? Not your forte? Let's see if the genres of the movie releases and the stock price have a relationship.

These ideas are all speculative, but I invite my readers to stay the course. The insights and graphs are at the bottom. Continue reading if you'd like.

In [ ]:
# Assuming the first movie listed is where this movie was filmed the most, we will replace the lists of movies
# with the first country and use that to find the top 10 countries that have movies are currently on Netflix 

movies_with_countries = netflix_movies.copy()
new=movies_with_countries['country'].str.split(',', n=5, expand=True)
countries = [('1st',0), ('2nd',1),('3rd',2),('4th',3),('5th',4),('6th',5)]
for i in countries:
    movies_with_countries[i[0]+" country"] = new[i[1]]
new_mwc = movies_with_countries.drop(columns=['show_id','type','director','cast'])
distribution_df = new_mwc[['1st country',
                                         'title']].groupby(
    '1st country'
).count().reset_index()
distribution_df.head()

We could say Hollywood knows how to make a good movie? Or should we say that one director is taking center stage and we just can't get enough?

In [ ]:
directors = movies_with_countries.copy()
directors = directors[directors['country']=='United States'][['director','title','rating','listed_in']]
directors.head()


new_dir = directors['director'].str.split(',', expand=True)
split = [('1st',0), ('2nd',1)]

# does genre matter?
for i in split:
    directors[i[0]+" Named Director"] = new_dir[i[1]]
top_directors = directors[['1st Named Director','title','rating','listed_in']].groupby(
    '1st Named Director'
)['title'].count().reset_index()


top_directors[top_directors['title']==top_directors.title.max()]

Let's be honest, Jay Karas is taking the market by storm with his work in comedy, so this one isn't too much of a surprise! Let's look for other trends that are in the data. Later, we will investigate if there is a correlation between the movies released and the stock price. We can't say conclusively that there is a direct cause and effect relationship; it's interested to find patterns in data, just for fun!

Now, we have to ask, *does the year a movie was released matter?* I can imagine that there have been times that the film industry took a hit and not too many movies reached production. We can only speculate about the greater implications. It could just mean that the most popular year had the best movies. We decide what this metric becomes in the end! 

In [ ]:
# # does genre matter?
# genre = []
# for i in genre:
#     directors[i[0]+" genre"] = genre[i[1]]
# top_directors = directors[['director','title','rating','listed_in']].groupby(
#     'director'
# )['title'].count().reset_index()
# top_directors['director'] = top_directors['director'].str.split(',')[0]

In [ ]:
titles_per_country = alt.Chart(distribution_df,
                              height=300,
                              width=500,
                              title='Number of Titles per Country').transform_window(
    rank='rank(title)',
    sort=[alt.SortField('title', order='descending')]
).transform_filter(
    alt.datum.rank <=10
).mark_bar(color='red').encode(
    y=alt.Y('title:Q',
           axis=alt.Axis(title='Number of Movies',
                         tickCount=4,
                         labelPadding=10,
                        titlePadding=10)),
    x=alt.X('1st country:N',
           sort=alt.EncodingSortField('title',order='descending'),
           title=None,
           axis=alt.Axis(
                        labelAngle=-45,
               labelFontSize=15
           )),
    color=alt.condition(
        alt.datum.rank == 1,
        alt.value('red'),
        alt.value('black')
    ),
    tooltip=['1st country:N', 'title']
)

titles_by_director = alt.Chart(top_directors,
                              height=300,
                              width=500,
                              title='Number of Titles by Director').transform_window(
    rank='rank(title)',
    sort=[alt.SortField('title', order='descending')]
).transform_filter(
    alt.datum.rank <=10
).mark_bar(color='red').encode(
    x=alt.X('1st Named Director:N',
           sort=alt.EncodingSortField('title',order='descending'),
           title=None,
           axis=alt.Axis(
                        labelAngle=-45,
               labelFontSize=15
           )),
    y=alt.Y('title:Q',
           axis=alt.Axis(title='Number of Movies',
                         tickCount=4,
                         labelPadding=10,
                        titlePadding=10)),
    color=alt.condition(
        alt.datum.rank == 1,
        alt.value('red'),
        alt.value('black')
    ),
    tooltip=['1st Named Director:N', 'title']
)

titles_by_year = alt.Chart(movies_over_time,
                              height=300,
                              width=500,
                              title='Number of Titles by Year').transform_window(
    rank='rank(title)',
    sort=[alt.SortField('title', order='descending')]
).transform_filter(
    alt.datum.rank <=10
).mark_bar(color='red').encode(
    y=alt.Y('title:Q',
           axis=alt.Axis(title='Number of Movies',
                         tickCount=4,
                         labelPadding=10,
                        titlePadding=10)),
    x=alt.X('release_year:N',
           sort=alt.EncodingSortField('title',order='descending'),
           title=None,
           axis=alt.Axis(
                        labelAngle=-45,
               labelFontSize=15
           )),
    color=alt.condition(
        alt.datum.rank == 1,
        alt.value('red'),
        alt.value('black')
    ),
    tooltip=['release_year:N', 'title']
)

In [ ]:
# Coding the graphs done in hidden cell. Data used in the graphs are visible in the cells above.
# Remake the year chart into something pertaining to actors, then continue with the genre scripting aboved
alt.vconcat((titles_per_country | titles_by_director),(titles_by_year))

In [ ]:
high_stock_price = alt.Chart(netflix_stocks,
                              height=300,
                              width=500,
                              title='Netflix Stock Price and Trading Volume over Time').mark_line(color='black').encode(
    y=alt.Y('High:Q',
           axis=alt.Axis(title='High Price',
                         # tickCount=4,
                         labelPadding=10,
                        titlePadding=10)),
    x=alt.X('Date:T',
           title='Date',
           axis=alt.Axis(
               labelAngle=-45,
               labelFontSize=15
           )),
    tooltip=[alt.Tooltip('Date:T'),alt.Tooltip('High:Q', format='2r')]
)

volume_of_stocks = alt.Chart(netflix_stocks,
                              height=300,
                              width=500,
                              title='Netflix Stock Price and Trading Volume over Time').mark_bar(color='red').encode(
    y=alt.Y('Volume:Q',
           axis=alt.Axis(title='Trading Volume',
                         # tickCount=4,
                         labelPadding=10,
                        titlePadding=10)),
    x=alt.X('Date:T',
           title='Date',
           axis=alt.Axis(
               labelAngle=-45,
               labelFontSize=15
           )),
    tooltip=[alt.Tooltip('Date:T'),alt.Tooltip('Volume:Q', format=',')])


# high_stock_price + volume_of_stocks
alt.layer(volume_of_stocks,high_stock_price).resolve_scale(y='independent')

As we can see, the stock price rose tremendously after 2016. Even though there was far less shares being traded at the time, the stock price kept driving higher. From the Number of Titles by Year and the Netflix Stock Price and Trading Volume over Time, we can see that 2017 was a pivotal year for the company. Let's take a look at just that segment of time for the movies release by date.

In [ ]:
netflix_movies[netflix_movies['release_year']==2017].groupby('rating').count()['title']

In [ ]:
movies_in_2017 = netflix_movies[netflix_movies['release_year']==2017][(netflix_movies['rating'].str.contains('TV-MA')) | (netflix_movies['rating'].str.contains('TV-14'))]
movies_in_2017['listed_in'] = movies_in_2017['listed_in'].str.split(',', expand=True)
movies_in_2017['country'] = movies_in_2017['country'].str.split(',', n=5, expand=True)
movies_in_2017

In [ ]:
movies_in_2017['duration'] = movies_in_2017['duration'].str.split(' ',expand=True)
movies_in_2017['duration'] = movies_in_2017['duration'].astype('int')
movies_in_2017 = movies_in_2017.rename(columns={'duration':'duration_min'})
m17_mean = movies_in_2017['duration_min'].mean()
top6_genres = movies_in_2017.groupby('listed_in').count()['title'].sort_values()[-6:].index.to_list()
movies_in_2017

In [ ]:
def agg_func(group):
    return group['description']

aggregate = pd.DataFrame(movies_in_2017[['listed_in','title','description']].groupby(['listed_in','title']).apply(agg_func))
aggregate.index = aggregate.index.droplevel(level=2)

In [ ]:
new_df=aggregate.groupby('listed_in').agg('count')

In [ ]:
aggregate.index.get_level_values(0).unique()

In [ ]:
aggregate 

In [ ]:
new_df.drop(columns=['count']).sort_values(by='description',ascending=False)

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
text = aggregate['description'].values 

wordcloud = WordCloud().generate(str(text))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

In [ ]:
import nltk
# Import packages and modules
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords') 
nltk.download('wordnet')

In [ ]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)
    
    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    return keywords


In [ ]:
# Create an instance of TfidfVectorizer
vectoriser = TfidfVectorizer(analyzer=preprocess_text)

# Fit to the data and transform to feature matrix
X_train = vectoriser.fit_transform(aggregate['description'])

# Convert sparse matrix to dataframe
X_train = pd.DataFrame.sparse.from_spmatrix(X_train)

# Save mapping on which index refers to which words
col_map = {v:k for k, v in vectoriser.vocabulary_.items()}
# Rename each column using the mapping
for col in X_train.columns:
    X_train.rename(columns={col: col_map[col]}, inplace=True)
X_train=X_train[X_train.columns[40:]]

In [ ]:
# Create an instance of RegexpTokenizer for alphanumeric tokens
tokeniser = RegexpTokenizer(r'\w+')

# Tokenise 'part1' string
tokens = tokeniser.tokenize(part1)
print(tokens)

In [ ]:

open_close_color = alt.condition("datum.Open <= datum.Close",
                                 alt.value("#06982d"),
                                 alt.value("#ae1325"))

stocks = netflix_stocks[netflix_stocks.Date > '2017-01-01'][netflix_stocks.Date < '2017-12-31']
stocks['month'] = stocks['Date'].dt.strftime('%m')

base = alt.Chart(stocks,
                 height=400,
                 width=800,
                 title='Netflix Stock Price and Trading Volume over Time').encode(
    x=alt.X('Date:T', 
            timeUnit='yearmonthdate', 
            #scale=alt.Scale(domain=domain),
            title='Dates of 2017',
            axis=alt.Axis(
                labelAngle=-45,
                labelFontSize=15
           ))
)

rule = base.mark_rule(x2Offset=10,
    xOffset=10).encode(
    alt.Y(
        'Low:Q',
        title='Price',
        scale=alt.Scale(zero=False)
    ),
    alt.Y2('High:Q'),
)

bar = base.mark_bar().encode(
    alt.Y('Open:Q'),
    alt.Y2('Close:Q'),
    color=open_close_color
)

chart =alt.layer(rule,bar).interactive()

In [ ]:
chart